# Project Report
![SEIR Model](/assets/readme/SEIR_normal.png "SEIR MODEL")
![Custom Model](/assets/readme/custom.png "Custom Model")
![Compared Models](/assets/readme/comparative_output.png "Comparitive")


In [2]:
%matplotlib inline
from ndlib.viz.bokeh.DiffusionTrend import DiffusionTrend
from assets.simulations import Simulation
from bokeh.io import output_notebook, show


output_notebook() # show bokeh in notebook
Sim = Simulation(node_number=5000, infection_rate=0.08, removal_rate=0.05, fraction_infected=0.1, time_simulated=180)
model, trends = Sim.SimpleSEIR(latent_period=0.14)  # 1 week latent period
model2, trends2 = Sim.CustomVaccineModel(vaccination_infection_rate=0.05)
show(DiffusionTrend(model, trends).plot())
show(DiffusionTrend(model2, trends2).plot())

Loading BokehJS ...

Running SimpleSEIR sim with 5000 nodes for 180 days
Running Custom sim with 5000 nodes for 180 days


# Preamble

## NDlib
This project uses [NDlib](https://ndlib.readthedocs.io/), which uses [Erdős–Rényi models](https://en.wikipedia.org/wiki/Erd%C5%91s%E2%80%93R%C3%A9nyi_model) as the base model we're running our simulations on.
On NDlib, there are 12 Epidemic models, of which 6 are Epidemic and 6 Opinion Dynamic:
Epidemic:
    * SI
    * SIS
    * SIR
    * SEIR
    * SEIS
    * SWIR
Opinion Dynamic:
    * Threshold
    * Generalised Threshold
    * Kertesz Threshold
    * Profile
    * Profile-Threshold
    * Independent Cascades

These Opinion Dynamics models are interesting and help the study of how opinions form in a population, however this is outside the scope of the project.

## Simulation inputs
All models share inputs of total nodes to run the simulation on, the fraction initially infected, and time the simulation is ran for. Based on the strain a user chooses, the percentage chance for infection and removal is obtained from [online data](https://www.dam.brown.edu/drp/talks/JosephSheely.pdf). In the absence of containing strategies, the infection is characterized by a high infection rate (we take α=0.9) and a low removal rate (γ=0.025 or 0.05). [Source](https://www.nature.com/articles/s41598-021-03721-x#Sec2)

We quickly realised that it was not feasible to create a graph with the actual values of population. Even 100,000 (1/5th of Bristol's population causes the program to freeze). Consequently, the population size is reduced and this factor of reduction is stored when later outputting statistics of model outputs.

In [1]:
# Measuring execution time for various graph sizes
import datetime
import networkx as nx

timings = {}
for i in (10**i for i in range(1,5)):
    start_time = datetime.datetime.now()
    nx.erdos_renyi_graph(i, 0.1)
    timings[i] = datetime.datetime.now() - start_time

for i in timings:
    print(i, ":", timings[i].total_seconds())

10 : 0.072951
100 : 0.001012
1000 : 0.105379
10000 : 9.447339


# Models
## Simple SEIR
Inputs:
* latent_period, which is the period before becoming infectious (E->I). Defaults to 0.33.
* removal_rate, which is the probability of nodes removal.
* infection_rate, which is the probability of infection when exposed.

This model, on one susceptible node coming into contact with an infected one, it will have the chance equal to infection_rate to become Exposed. On each iteration as Exposed, there is a chance equal to latent_period to become Infected. Each iteration as Infected, there is a chance equal to removal_rate to become Removed.

Susceptible -> Exposed -> Infected -> Removed

## Dynamic SIR
Inputs:
* removal_rate, which is the probability of nodes removal.
* infection_rate, which is the probability of infection when exposed.

This model is comparitively simpler, with no latent_period. That being said, this model uses Dynamic Network Modelling where network topology evolves as time goes by. We're using a Snapshot Network, as we can separate network history into a series of snapshots, one a day. An alternative would be an Interaction Network, however this is only used when interaction is constant/cannot be separated.

Model takes a WHILE to run!

## Custom Vaccination Model
To introduce vaccination effects, develop our own custom model! This will have multiple transfers

Susceptible -> Exposed -> Infectious -> Dead or Recover -> Susceptible (after 3 months)
Susceptible -> Vaccinatied -> Susceptible (after 3 months)

Transitions from susceptible to exposed is based on probability once come in contact with an Infectious. From Exposed to Infectous, there is a change of 0.3485, since after 7 iterations the percentage chance is =1-(POWER(1-0.35,7))=0.950180319. From Infectious to Recovered, the chance 0.193, since after 14 iterations the percentage chance is =1-(POWER(1-0.193,14))=0.95031452. To code the custom class, NodeStochastic methods were used, which operate on a probability on whether the node transitions state. c1 also uses a trigger_state which ensures it is only triggered on contact with an Infectious person. Lastly, both vaccinated and recovered people become susceptible again after 84 days (simulating fading of immunity)

In South West of england, 82% are fully vaccinated (https://www.bbc.co.uk/news/health-55274833).

# Variable choices
We used different variables for various historical strains of covid, which were found [here](https://www.thelancet.com/pdfs/journals/laninf/PIIS1473-3099(21)00648-4.pdf).

50 // 602 = 0.08 for pre alpha

71// 602 = 0.11 for delta 